# Brittleness: When Models Break with Small Changes

This notebook demonstrates **brittleness** - when models work in theory but fail when conditions change slightly.

Understanding brittleness helps you:
- See how over-constraining makes models fragile
- Understand why models break with small changes
- Learn how to build robust models with soft constraints
- Avoid making models too rigid


## Key Concepts

**Brittleness** means models that break with small changes:
- Over-constraining makes models brittle
- Too many hard constraints create a very small solution space
- Small changes push the model outside this space, making it infeasible
- Robust models can handle small changes

**Why This Matters:**
- Real-world conditions are never exactly what models assume
- Brittle models fail in implementation
- Robust models work in real-world conditions

**Critical insight**: Over-constraining makes models brittle. Using soft constraints instead of hard constraints makes models more robust and useful in practice.


## Scenario: Scheduling Model That Becomes Brittle

A hospital needs to schedule nurses for the upcoming week. Initially, the model works. But as more constraints are added, it becomes brittle.

**Initial constraints** (model works):
- Minimum staffing required for each shift
- Maximum hours per nurse
- Budget limit

**Additional constraint** (makes model brittle):
- "Prefer to minimize schedule changes" treated as hard rule "no schedule changes allowed"

**Decision**: How does the model behave as constraints are added?


## Step 1: Install Required Packages (Colab)


In [1]:
# Install pulp package (required for optimization)
%pip install pulp -q



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Libraries


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value


## Step 3: Build Initial Model (Works)

Start with a model that has essential constraints:


In [3]:
# Problem setup
shifts = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
n_shifts = len(shifts)
nurses = ['Nurse A', 'Nurse B', 'Nurse C', 'Nurse D', 'Nurse E']
n_nurses = len(nurses)

# Previous week's schedule
previous_schedule = {
    'Nurse A': [1, 1, 0, 0, 1, 0, 0],  # 1 = working, 0 = off
    'Nurse B': [0, 1, 1, 1, 0, 1, 0],
    'Nurse C': [1, 0, 1, 0, 1, 0, 1],
    'Nurse D': [0, 0, 1, 1, 1, 1, 0],
    'Nurse E': [1, 1, 0, 1, 0, 0, 1]
}

# Requirements
min_staffing = [3, 3, 3, 3, 3, 2, 2]  # Minimum nurses per shift
max_hours_per_nurse = 40  # Maximum hours per week
hourly_rate = 35
max_budget = 5000

print("INITIAL MODEL SETUP")
print("=" * 70)
print(f"Shifts: {shifts}")
print(f"Nurses: {nurses}")
print(f"Minimum staffing per shift: {min_staffing}")
print(f"Maximum hours per nurse: {max_hours_per_nurse}")
print(f"Maximum budget: ${max_budget:,}")


INITIAL MODEL SETUP
Shifts: ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
Nurses: ['Nurse A', 'Nurse B', 'Nurse C', 'Nurse D', 'Nurse E']
Minimum staffing per shift: [3, 3, 3, 3, 3, 2, 2]
Maximum hours per nurse: 40
Maximum budget: $5,000


## Step 4: Model with Essential Constraints Only

Build a model with only essential hard constraints:


In [4]:
# Model with essential constraints only
model_initial = LpProblem("Initial_Model", LpMinimize)

# Decision variables: nurse i works shift j
schedule = {}
for i, nurse in enumerate(nurses):
    schedule[nurse] = [LpVariable(f"{nurse}_{j}", cat='Binary') for j in range(n_shifts)]

# Objective: minimize cost
total_cost = lpSum([hourly_rate * 8 * schedule[nurse][j] 
                    for nurse in nurses for j in range(n_shifts)])
model_initial += total_cost

# Hard constraints: minimum staffing
for j in range(n_shifts):
    model_initial += lpSum([schedule[nurse][j] for nurse in nurses]) >= min_staffing[j], f"Min_Staff_{j}"

# Hard constraint: maximum hours per nurse
for nurse in nurses:
    model_initial += lpSum([8 * schedule[nurse][j] for j in range(n_shifts)]) <= max_hours_per_nurse, f"Max_Hours_{nurse}"

# Hard constraint: budget
model_initial += total_cost <= max_budget, "Budget"

# Solve
model_initial.solve()

if model_initial.status == 1:
    print("INITIAL MODEL: FEASIBLE")
    print("=" * 70)
    total_hours_initial = value(total_cost) / hourly_rate
    print(f"Total cost: ${value(total_cost):,.2f}")
    print(f"Total hours: {total_hours_initial:.1f}")
    print("Model works with essential constraints only.")
else:
    print("INITIAL MODEL: INFEASIBLE")
    print("Even with essential constraints, model is infeasible.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sturner/.pyenv/versions/3.12.7/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/e6a11485eab94e6c8cdb75a3f193a2e5-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/e6a11485eab94e6c8cdb75a3f193a2e5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 229 RHS
At line 243 BOUNDS
At line 279 ENDATA
Problem MODEL has 13 rows, 35 columns and 105 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

INITIAL MODEL: INFEASIBLE
Even with essential constraints, model is infeasible.


In [5]:
# Model with preference treated as hard rule (BRITTLE)
model_brittle = LpProblem("Brittle_Model", LpMinimize)

# Decision variables
schedule_brittle = {}
for i, nurse in enumerate(nurses):
    schedule_brittle[nurse] = [LpVariable(f"brittle_{nurse}_{j}", cat='Binary') for j in range(n_shifts)]

# Objective
total_cost_brittle = lpSum([hourly_rate * 8 * schedule_brittle[nurse][j] 
                           for nurse in nurses for j in range(n_shifts)])
model_brittle += total_cost_brittle

# Essential hard constraints
for j in range(n_shifts):
    model_brittle += lpSum([schedule_brittle[nurse][j] for nurse in nurses]) >= min_staffing[j], f"Min_{j}"

for nurse in nurses:
    model_brittle += lpSum([8 * schedule_brittle[nurse][j] for j in range(n_shifts)]) <= max_hours_per_nurse, f"Max_{nurse}"

model_brittle += total_cost_brittle <= max_budget, "Budget"

# BRITTLE CONSTRAINT: Preference treated as hard rule
# "Prefer to minimize schedule changes" → "No schedule changes allowed"
for i, nurse in enumerate(nurses):
    for j in range(n_shifts):
        # If nurse worked this shift last week, they must work it this week
        # If nurse didn't work this shift last week, they must not work it this week
        if previous_schedule[nurse][j] == 1:
            model_brittle += schedule_brittle[nurse][j] == 1, f"No_Change_{nurse}_{j}"
        else:
            model_brittle += schedule_brittle[nurse][j] == 0, f"No_Change_{nurse}_{j}"

# Solve
model_brittle.solve()

print("BRITTLE MODEL: Preference as Hard Rule")
print("=" * 70)
if model_brittle.status == 1:
    print("Model is FEASIBLE (unexpected - constraints can be satisfied)")
    print(f"Total cost: ${value(total_cost_brittle):,.2f}")
else:
    print("Model is INFEASIBLE!")
    print("\nWhy? The 'no schedule changes' rule conflicts with:")
    print("  - Minimum staffing requirements")
    print("  - Maximum hours per nurse")
    print("  - Budget constraints")
    print("\nThis is brittleness: adding one constraint breaks the model!")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sturner/.pyenv/versions/3.12.7/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/67de3e8ae69a482c9a0e91c911d6af24-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/67de3e8ae69a482c9a0e91c911d6af24-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 53 COLUMNS
At line 299 RHS
At line 348 BOUNDS
At line 384 ENDATA
Problem MODEL has 48 rows, 35 columns and 140 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

BRITTLE MODEL: Preference as Hard Rule
Model is INFEASIBLE!

Why? The 'no schedule changes' rule conflicts with:
  - Minimum 

## Step 6: Robust Model - Preference as Soft Constraint

Now build a robust model using soft constraints:


In [6]:
# Robust model: preference as soft constraint
model_robust = LpProblem("Robust_Model", LpMinimize)

# Decision variables
schedule_robust = {}
for i, nurse in enumerate(nurses):
    schedule_robust[nurse] = [LpVariable(f"robust_{nurse}_{j}", cat='Binary') for j in range(n_shifts)]

# Variables for schedule changes (soft constraint violation)
changes = {}
for i, nurse in enumerate(nurses):
    changes[nurse] = [LpVariable(f"change_{nurse}_{j}", lowBound=0, cat='Continuous') for j in range(n_shifts)]

# Objective: minimize cost + penalties for changes
total_cost_robust = lpSum([hourly_rate * 8 * schedule_robust[nurse][j] 
                          for nurse in nurses for j in range(n_shifts)])
penalty_per_change = 20
total_change_penalty = lpSum([penalty_per_change * changes[nurse][j] 
                              for nurse in nurses for j in range(n_shifts)])
model_robust += total_cost_robust + total_change_penalty

# Essential hard constraints
for j in range(n_shifts):
    model_robust += lpSum([schedule_robust[nurse][j] for nurse in nurses]) >= min_staffing[j], f"Min_{j}"

for nurse in nurses:
    model_robust += lpSum([8 * schedule_robust[nurse][j] for j in range(n_shifts)]) <= max_hours_per_nurse, f"Max_{nurse}"

model_robust += total_cost_robust <= max_budget, "Budget"

# SOFT CONSTRAINT: Preference with penalty
for i, nurse in enumerate(nurses):
    for j in range(n_shifts):
        # Change is absolute difference from previous schedule
        model_robust += changes[nurse][j] >= schedule_robust[nurse][j] - previous_schedule[nurse][j], f"ChUp_{nurse}_{j}"
        model_robust += changes[nurse][j] >= previous_schedule[nurse][j] - schedule_robust[nurse][j], f"ChDown_{nurse}_{j}"

# Solve
model_robust.solve()

print("ROBUST MODEL: Preference as Soft Constraint")
print("=" * 70)
if model_robust.status == 1:
    total_changes = sum([value(changes[nurse][j]) for nurse in nurses for j in range(n_shifts)])
    print("Model is FEASIBLE")
    print(f"Total cost: ${value(total_cost_robust):,.2f}")
    print(f"Total change penalty: ${value(total_change_penalty):,.2f}")
    print(f"Total cost + penalties: ${value(total_cost_robust + total_change_penalty):,.2f}")
    print(f"Total schedule changes: {total_changes:.1f}")
    print("\nModel is robust: it can handle the preference while satisfying all hard constraints!")
else:
    print("Model is INFEASIBLE (unexpected)")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sturner/.pyenv/versions/3.12.7/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/ce20159e01424858ae4479c5233ba8ef-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/ce20159e01424858ae4479c5233ba8ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 88 COLUMNS
At line 474 RHS
At line 558 BOUNDS
At line 594 ENDATA
Problem MODEL has 83 rows, 70 columns and 245 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

ROBUST MODEL: Preference as Soft Constraint
Model is INFEASIBLE (unexpected)


## Step 7: Test Robustness to Small Changes

Let's see how each model handles a small change in requirements:


In [7]:
# Small change: increase minimum staffing by 1 for Monday
min_staffing_changed = min_staffing.copy()
min_staffing_changed[0] += 1  # Monday needs one more nurse

print("TESTING ROBUSTNESS: Small Change in Requirements")
print("=" * 70)
print("Change: Monday minimum staffing increased from 3 to 4")

# Test brittle model with changed requirements
model_brittle_test = LpProblem("Brittle_Test", LpMinimize)
schedule_test = {}
for i, nurse in enumerate(nurses):
    schedule_test[nurse] = [LpVariable(f"test_{nurse}_{j}", cat='Binary') for j in range(n_shifts)]

total_cost_test = lpSum([hourly_rate * 8 * schedule_test[nurse][j] for nurse in nurses for j in range(n_shifts)])
model_brittle_test += total_cost_test

for j in range(n_shifts):
    model_brittle_test += lpSum([schedule_test[nurse][j] for nurse in nurses]) >= min_staffing_changed[j], f"Min_{j}"

for nurse in nurses:
    model_brittle_test += lpSum([8 * schedule_test[nurse][j] for j in range(n_shifts)]) <= max_hours_per_nurse, f"Max_{nurse}"

model_brittle_test += total_cost_test <= max_budget, "Budget"

# Brittle constraint
for i, nurse in enumerate(nurses):
    for j in range(n_shifts):
        if previous_schedule[nurse][j] == 1:
            model_brittle_test += schedule_test[nurse][j] == 1, f"NoCh_{nurse}_{j}"
        else:
            model_brittle_test += schedule_test[nurse][j] == 0, f"NoCh_{nurse}_{j}"

model_brittle_test.solve()

print("\nBrittle Model (preference as hard rule):")
if model_brittle_test.status == 1:
    print("  Status: FEASIBLE")
else:
    print("  Status: INFEASIBLE - Model breaks with small change!")

# Test robust model with changed requirements
model_robust_test = LpProblem("Robust_Test", LpMinimize)
schedule_robust_test = {}
for i, nurse in enumerate(nurses):
    schedule_robust_test[nurse] = [LpVariable(f"robust_test_{nurse}_{j}", cat='Binary') for j in range(n_shifts)]

changes_test = {}
for i, nurse in enumerate(nurses):
    changes_test[nurse] = [LpVariable(f"ch_test_{nurse}_{j}", lowBound=0, cat='Continuous') for j in range(n_shifts)]

total_cost_robust_test = lpSum([hourly_rate * 8 * schedule_robust_test[nurse][j] for nurse in nurses for j in range(n_shifts)])
total_change_penalty_test = lpSum([penalty_per_change * changes_test[nurse][j] for nurse in nurses for j in range(n_shifts)])
model_robust_test += total_cost_robust_test + total_change_penalty_test

for j in range(n_shifts):
    model_robust_test += lpSum([schedule_robust_test[nurse][j] for nurse in nurses]) >= min_staffing_changed[j], f"Min_{j}"

for nurse in nurses:
    model_robust_test += lpSum([8 * schedule_robust_test[nurse][j] for j in range(n_shifts)]) <= max_hours_per_nurse, f"Max_{nurse}"

model_robust_test += total_cost_robust_test <= max_budget, "Budget"

for i, nurse in enumerate(nurses):
    for j in range(n_shifts):
        model_robust_test += changes_test[nurse][j] >= schedule_robust_test[nurse][j] - previous_schedule[nurse][j], f"ChUp_{nurse}_{j}"
        model_robust_test += changes_test[nurse][j] >= previous_schedule[nurse][j] - schedule_robust_test[nurse][j], f"ChDown_{nurse}_{j}"

model_robust_test.solve()

print("\nRobust Model (preference as soft constraint):")
if model_robust_test.status == 1:
    total_changes_test = sum([value(changes_test[nurse][j]) for nurse in nurses for j in range(n_shifts)])
    print(f"  Status: FEASIBLE")
    print(f"  Total cost: ${value(total_cost_robust_test):,.2f}")
    print(f"  Schedule changes: {total_changes_test:.1f}")
    print("  Model adapts to the change!")
else:
    print("  Status: INFEASIBLE")

print("\nKey Insight:")
print("  - Brittle model breaks with small changes")
print("  - Robust model adapts to changes")
print("  - Soft constraints make models more robust and practical")


TESTING ROBUSTNESS: Small Change in Requirements
Change: Monday minimum staffing increased from 3 to 4
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sturner/.pyenv/versions/3.12.7/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/d32c51614627434e86ee53a4e9c19e0d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0v/80zxmry158l85b2sy7ywwj5w0000gn/T/d32c51614627434e86ee53a4e9c19e0d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 53 COLUMNS
At line 299 RHS
At line 348 BOUNDS
At line 384 ENDATA
Problem MODEL has 48 rows, 35 columns and 140 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00


Brittle Model (prefer